In [26]:
#translation

import cv2
import numpy as np
img = cv2.imread('sakura.jpg')
num_rows, num_cols = img.shape[:2] #choose first two data
print(img.shape, img.dtype)

#image translation using translation_matrix
translation_matrix = np.float32([ [0.2, 0.5,  70], 
                                  [0.5, 0.2, 110] ])

img_translation1 = cv2.warpAffine(img, translation_matrix, (num_cols+70, num_rows+110), cv2.INTER_LINEAR)
#cv2.warpAffine(src, M(2*3 mat), dsize[, dst[, flags[, borderMode[, borderValue]]]]) → dst

#image translation using ROI
img_translation2 = np.zeros((num_rows+110, num_cols+70, 3), dtype=np.uint8) 
print(img_translation2.shape)
img_translation2[110:num_rows+110, 70:num_cols+70, :] = img

cv2.imshow('Translation1', img_translation1)
cv2.imshow('Translation2', img_translation2)
cv2.waitKey()
cv2.destroyAllWindows()

(225, 225, 3) uint8
(335, 295, 3)


In [31]:
#rotation

import cv2
import numpy as np
img = cv2.imread('sakura.jpg')
num_rows, num_cols = img.shape[:2]
rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), #centrePt
                                          30, #rotationDeg ClockWise
                                          0.7) #resizeRatio
print(rotation_matrix)
img_rotation = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
cv2.imshow('Rotation', img_rotation)
cv2.imshow('Original', img)
cv2.waitKey()
cv2.destroyAllWindows()

[[ 0.60621778  0.35        4.92549945]
 [-0.35        0.60621778 83.67549945]]


In [43]:
#scaling
#http://paulbourke.net/miscellaneous/interpolation/

import cv2
img = cv2.imread('sakura.jpg')
img_scaled = cv2.resize(img,None,fx=1.5, fy=1.5, interpolation = cv2.INTER_LINEAR)
cv2.imshow('Scaling - Linear Interpolation', img_scaled)
img_scaled = cv2.resize(img,None,fx=1.5, fy=1.5, interpolation = cv2.INTER_CUBIC)
cv2.imshow('Scaling - Cubic Interpolation', img_scaled)

#specify new (col, row) in opencv, but in np (col, row)
img_scaled = cv2.resize(img,(600, 400), interpolation = cv2.INTER_AREA)
cv2.imshow('Scaling - Skewed Size', img_scaled)
cv2.waitKey()
cv2.destroyAllWindows()

In [59]:
#perspective transformation
#Practice
import cv2
import numpy as np
wall = cv2.imread('tile_texture_4.jpg')
floor = cv2.imread('peekaboo.png')
rows, cols = wall.shape[:2]
frows, fcols = floor.shape[:2]
newRows, newCols = 640, 640

#in numpy, specify in the order of col, row
#North
src_points = np.float32([[0     ,      0], 
                         [cols-1,      0], 
                         [0     , rows-1], 
                         [cols-1, rows-1]])

Ndst_points = np.float32([[0       ,   0], 
                         [newCols-1,   0], 
                         [220      , 220], 
                         [420      , 220]])

#South
Sdst_points = np.float32([[220      ,       420], 
                          [420      ,       420],
                          [0       , newRows-1], 
                          [newCols-1, newRows-1]])

#East
Edst_points = np.float32([[0  ,         0], 
                          [0  , newRows-1], 
                          [220,       220], 
                          [220,       420]])

#West
Wdst_points = np.float32([[420     ,       220], 
                         [newCols-1,         0], 
                         [420      ,       420], 
                         [newCols-1, newRows-1]])

#Floor
fsrc_points = np.float32([[0      ,       0], 
                          [fcols-1,       0], 
                          [0      , frows-1], 
                          [fcols-1, frows-1]])

fdst_points = np.float32([[221       ,   221], 
                          [419,   221], 
                          [221      , 419], 
                          [419      , 419]])

#cv2.findHomography() can accept more than 4 pairs of corresponding points
Nprojective_matrix = cv2.getPerspectiveTransform(src_points, Ndst_points)
Sprojective_matrix = cv2.getPerspectiveTransform(src_points, Sdst_points)
Eprojective_matrix = cv2.getPerspectiveTransform(src_points, Edst_points)
Wprojective_matrix = cv2.getPerspectiveTransform(src_points, Wdst_points)
fprojective_matrix = cv2.getPerspectiveTransform(fsrc_points, fdst_points)

Nwall = cv2.warpPerspective(wall, Nprojective_matrix, (640,640))
Swall = cv2.warpPerspective(wall, Sprojective_matrix, (640,640))
Ewall = cv2.warpPerspective(wall, Eprojective_matrix, (640,640))
Wwall = cv2.warpPerspective(wall, Wprojective_matrix, (640,640))
floor = cv2.warpPerspective(floor, fprojective_matrix, (640,640))

temp1 = cv2.add(Nwall, Swall)
temp2 = cv2.add(Ewall, Wwall)
walls = cv2.add(temp1, temp2)
room = cv2.add(walls, floor)

ret, masked_room = cv2.threshold(room, 150, 0, cv2.THRESH_TOZERO)

rec_mask = np.zeros_like(masked_room)
rec_mask[:,:,:] = 255
cv2.rectangle(rec_mask, (220, 220), (420, 420), (0, 255, 255), 15)
masked_room = cv2.bitwise_and(masked_room, rec_mask)

cv2.imshow('Output', room)
cv2.imshow('Masked Output', masked_room)

cv2.waitKey()
cv2.destroyAllWindows()